In [1]:
# Import necessary libraries/packages
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error

# Fix random seed
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [2]:
# Read data from csv file
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
le = LabelEncoder()
sc = StandardScaler()

In [4]:
# Preprocess data
print(data.shape)
data = data.drop(data[(data.year > 2030) | (data.transmission == 2) | (data.fueltype == 1)].index)
print(data.shape)

x = data.drop('price', axis=1)
x = x.drop('ID', axis=1)
test = test.drop('ID', axis=1)
x = x.drop('tax', axis=1)
test = test.drop('tax', axis=1)

y = data['price'].values

data['model'] = le.fit_transform(data['model'])
x['model'] = le.transform(x['model'])
test['model'] = le.transform(test['model'])

x = sc.fit_transform(x)
test = sc.transform(test)

(34170, 11)
(34166, 11)


In [5]:
def _predict(n, train_set, train_ans, result:list, weights):
    knn = KNeighborsRegressor(n_neighbors=n, weights=weights)
    knn.fit(train_set, train_ans)
    train_pred = knn.predict(train_set)
    
    current = {
        'n': n,
        'weights': weights,
        'Train': mean_absolute_error(train_ans, train_pred),
        }
    result.append(current)

In [8]:
result = []
for i in range (3, 19, 2):
    _predict(i, x, y, result, 'uniform')
for i in range (3, 19, 2):
    _predict(i, x, y, result, 'distance')
    
pd.DataFrame(result).sort_values(by=['Train'], ascending=True)

,n,weights,Train
10,7,distance,19.970358
13,13,distance,19.977352
14,15,distance,19.977352
15,17,distance,19.977352
12,11,distance,19.980396
11,9,distance,19.984793
9,5,distance,19.987600
8,3,distance,20.141603
0,3,uniform,935.369032
1,5,uniform,1093.089130


In [ ]:
knn = KNeighborsRegressor(n_neighbors=7, weights='distance')
knn.fit(x, y)
res = knn.predict(test)
res = pd.DataFrame(data=res, columns=['price'])

res.to_csv('submit.csv', index_label='id')